In [2]:

%matplotlib inline

import pandas as pd
import sklearn
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
import random
mpl.pyplot.style.use('ggplot')
from graphviz import Digraph

In [3]:
loandata = pd.read_csv('../data/LoanStats3d.csv',skiprows=[0])

C:\Users\Dom\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
completedloans=loandata[(loandata.loan_status=='Charged Off')|(loandata.loan_status=='Fully Paid')]

In [5]:
# Split into test and training sets. Allocating 80% for training data
random.seed(1234)
traindata, testdata = train_test_split(completedloans, test_size = 0.2)

## Data Cleaning


In [6]:
traindata.drop(['id','member_id','funded_amnt','emp_title','desc','url','pymnt_plan','title'],1,inplace=True)
#traindata.drop('id',1,inplace=True)


C:\Users\Dom\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [7]:
from datetime import datetime
traindata.earliest_cr_line = pd.to_datetime(traindata.earliest_cr_line)
traindata.last_pymnt_d = pd.to_datetime(traindata.last_pymnt_d)
traindata.next_pymnt_d = pd.to_datetime(traindata.next_pymnt_d)
traindata.last_credit_pull_d = pd.to_datetime(traindata.last_credit_pull_d)

C:\Users\Dom\Anaconda2\lib\site-packages\pandas\core\generic.py:2698: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [8]:
traindata.earliest_cr_line =[datetime.toordinal(x) for x in traindata.earliest_cr_line]
traindata.last_pymnt_d =[datetime.toordinal(x) for x in traindata.last_pymnt_d]
traindata.next_pymnt_d =[datetime.toordinal(x) for x in traindata.next_pymnt_d]
traindata.last_credit_pull_d =[datetime.toordinal(x) for x in traindata.last_credit_pull_d]

In [9]:
traindata.int_rate = pd.Series(traindata.int_rate).str.replace('%', '').astype(float)
traindata.revol_util= pd.Series(traindata.revol_util).str.replace('%', '').astype(float)

In [10]:
testdata.earliest_cr_line = pd.to_datetime(testdata.earliest_cr_line)
testdata.last_pymnt_d = pd.to_datetime(testdata.last_pymnt_d)
testdata.next_pymnt_d = pd.to_datetime(testdata.next_pymnt_d)
testdata.last_credit_pull_d = pd.to_datetime(testdata.last_credit_pull_d)
testdata.earliest_cr_line =[datetime.toordinal(x) for x in testdata.earliest_cr_line]
testdata.last_pymnt_d =[datetime.toordinal(x) for x in testdata.last_pymnt_d]
testdata.next_pymnt_d =[datetime.toordinal(x) for x in testdata.next_pymnt_d]
testdata.last_credit_pull_d =[datetime.toordinal(x) for x in testdata.last_credit_pull_d]
testdata.int_rate = pd.Series(testdata.int_rate).str.replace('%', '').astype(float)
testdata.revol_util= pd.Series(testdata.revol_util).str.replace('%', '').astype(float)

In [11]:
# Import SciKit Learn functions
from sklearn.metrics import roc_curve, auc , roc_auc_score, confusion_matrix, mean_absolute_error
from sklearn.tree import DecisionTreeClassifier , export_graphviz
from sklearn.cross_validation import train_test_split
from sklearn.externals.six import StringIO
from IPython.display import Image
from sklearn.feature_extraction import DictVectorizer
from sklearn import preprocessing
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

In [12]:
# Rearranging the columns so that the response variable is at index 0
cols = traindata.columns.tolist()
cols =(cols[12:13]+cols[0:12]+cols[13:])
traindata = traindata[cols]

In [13]:
testdata = testdata[cols]

In [14]:
# This code transforms the string columns into numerical values 
le = preprocessing.LabelEncoder()
le.fit(traindata.home_ownership)
le.classes_
traindata.home_ownership=le.transform(traindata.home_ownership)

le.fit(traindata.term)
le.classes_
traindata.term=le.transform(traindata.term)

le.fit(traindata.grade)
le.classes_
traindata.grade=le.transform(traindata.grade)


le.fit(traindata.loan_status)
le.classes_
traindata.loan_status=le.transform(traindata.loan_status)

le.fit(traindata.emp_length)
le.classes_
traindata.emp_length=le.transform(traindata.emp_length)

le.fit(traindata.verification_status)
le.classes_
traindata.verification_status=le.transform(traindata.verification_status)

le.fit(traindata.issue_d)
le.classes_
traindata.issue_d=le.transform(traindata.issue_d)

le.fit(traindata.purpose)
le.classes_
traindata.purpose=le.transform(traindata.purpose)

le.fit(traindata.zip_code)
le.classes_
traindata.zip_code=le.transform(traindata.zip_code)

le.fit(traindata.addr_state)
le.classes_
traindata.addr_state=le.transform(traindata.addr_state)

le.fit(traindata.initial_list_status)
le.classes_
traindata.initial_list_status=le.transform(traindata.initial_list_status)

le.fit(traindata.application_type)
le.classes_
traindata.application_type=le.transform(traindata.application_type)

le.fit(traindata.verification_status_joint)
le.classes_
traindata.verification_status_joint=le.transform(traindata.verification_status_joint)


le.fit(traindata.sub_grade)
le.classes_
traindata.sub_grade=le.transform(traindata.sub_grade)
# Split the target from the input variables
X_train = traindata.iloc[:, 1:]
y_train = traindata.iloc[:, 0]

C:\Users\Dom\Anaconda2\lib\site-packages\numpy\lib\arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))
C:\Users\Dom\Anaconda2\lib\site-packages\numpy\lib\arraysetops.py:259: FutureWarning: numpy equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  return aux[:-1][aux[1:] == aux[:-1]]


In [15]:
# This code transforms the string columns into numerical values 
le = preprocessing.LabelEncoder()
le.fit(testdata.home_ownership)
le.classes_
testdata.home_ownership=le.transform(testdata.home_ownership)

le.fit(testdata.term)
le.classes_
testdata.term=le.transform(testdata.term)

le.fit(testdata.grade)
le.classes_
testdata.grade=le.transform(testdata.grade)


le.fit(testdata.loan_status)
le.classes_
testdata.loan_status=le.transform(testdata.loan_status)

le.fit(testdata.emp_length)
le.classes_
testdata.emp_length=le.transform(testdata.emp_length)

le.fit(testdata.verification_status)
le.classes_
testdata.verification_status=le.transform(testdata.verification_status)

le.fit(testdata.issue_d)
le.classes_
testdata.issue_d=le.transform(testdata.issue_d)

le.fit(testdata.purpose)
le.classes_
testdata.purpose=le.transform(testdata.purpose)

le.fit(testdata.zip_code)
le.classes_
testdata.zip_code=le.transform(testdata.zip_code)

le.fit(testdata.addr_state)
le.classes_
testdata.addr_state=le.transform(testdata.addr_state)

le.fit(testdata.initial_list_status)
le.classes_
testdata.initial_list_status=le.transform(testdata.initial_list_status)

le.fit(testdata.application_type)
le.classes_
testdata.application_type=le.transform(testdata.application_type)

le.fit(testdata.verification_status_joint)
le.classes_
testdata.verification_status_joint=le.transform(testdata.verification_status_joint)


le.fit(testdata.sub_grade)
le.classes_
testdata.sub_grade=le.transform(testdata.sub_grade)
# Split the target from the input variables
X_test = testdata.iloc[:, 1:]
y_test = testdata.iloc[:, 0]

In [16]:
len(X_train.index)

32057

In [17]:
nullcolls=(X_train.count()!=len(X_train.index))
matches = [i for i in range(0,len(nullcolls)) if i == True]
names=nullcolls[nullcolls]
nadroplist=list(names.index)

In [18]:
X_train.drop(nadroplist,1,inplace=True)
X_test.drop(nadroplist,1,inplace=True)

In [19]:
rfclf = RandomForestClassifier(n_estimators=100, max_features='auto', oob_score=True, random_state=1)


# Fit the random forest classifier
rfclf.fit(X_train, y_train)



RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=True, random_state=1, verbose=0, warm_start=False)

In [31]:
# compute the feature importances
importances=pd.DataFrame({'feature':X_train.columns, 'importance':rfclf.feature_importances_})

importances.sort_index(by='importance',ascending=False)

C:\Users\Dom\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


,feature,importance
28,total_rec_prncp,0.219900
34,last_pymnt_amnt,0.189716
26,total_pymnt,0.149859
27,total_pymnt_inv,0.139398
0,loan_amnt,0.046831
31,recoveries,0.044018
32,collection_recovery_fee,0.036445
4,installment,0.030878
1,funded_amnt_inv,0.030254
33,last_pymnt_d,0.028356


In [27]:
# compute the out-of-bag classification accuracy
rfclf.oob_score_

0.99956327791122068

In [22]:
mean_absolute_error(y_test, rfclf.predict(X_test))

0.00087336244541484718

In [23]:
probs = rfclf.predict_proba(X_test)[:,1]
metrics.roc_auc_score(y_test,probs)

0.99996581269241747

In [24]:
metrics.accuracy_score(y_test,rfclf.predict(X_test))

0.9991266375545852

In [25]:
pd.crosstab(y_test, rfclf.predict(X_test), rownames=['actual'], colnames=['predicted'])

predicted,0,1
actual,,
0,1203,7
1,0,6805


In [26]:
from sklearn.cross_validation import cross_val_score
cross_val_score(rfclf, X_train, y_train, cv=10, scoring='roc_auc').mean()

KeyboardInterrupt: 